In [1]:
import os, sys
from dask import compute, persist
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split,KFold,RandomizedSearchCV, GridSearchCV ,HyperbandSearchCV#, RandomizedSearchCV
import matplotlib.pyplot as plt
import dask_xgboost as dxgb
import xgboost as xgb
import pandas as pd
# from sklearn.model_selection import KFold,RandomizedSearchCV 
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,matthews_corrcoef

In [2]:
import joblib

In [3]:
client = Client()

In [4]:
client

Client Scheduler: tcp://127.0.0.1:49179 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [5]:
# ddf = dd.read_csv("data/184B5.csv")

df = []
for x in os.listdir("data"):
    print (x)
    df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df.append(df_temp)

184B5.csv
BT483.csv
HCC1428.csv
HCC1806.csv
HCC202.csv
Hs578T.csv
MCF12A.csv
MDAMB231.csv
MDAMB468.csv
SKBR3.csv
UACC3199.csv
ZR751.csv


In [6]:
ddf = dd.concat(df,axis=0)

In [7]:
print ("filling NA")
# y = y.compute()
with ProgressBar():
    for m in ["p.HER2","p.PLCg2"] :
#         print (df[m])
        
        ddf["%s_c"%(m)] =ddf[m].fillna(ddf[m].mean() )#, inplace=True )

filling NA


In [8]:
with ProgressBar():
    df2 = dd.get_dummies(ddf.categorize()).persist()

In [9]:
# df2.columns

In [10]:
# my_dummies = dd.get_dummies(ddf["time","treatment"])
# my_dummies_treatment = dd.get_dummies(ddf["treatment"]).persist()
# df2 = df2.drop(["p.HER2","p.PLCg2","cellID","fileID"],axis=1)
df2 = df2.drop(["p.HER2","p.PLCg2"],axis=1)

In [11]:
# my_dummies_time = dd.get_dummies(ddf["time"]).persist()

In [12]:
# df_dummies = dd.concat([my_dummies_treatment,my_dummies_time],axis=1)

In [13]:
rounds = {}
genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']
for x in range(len (genes)):
#     print (x)
    my_list = ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK',   'p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']
    if x+1 <len (genes):
#         my_list = genes 
#         print (x,genes[x], genes[x+1])
        my_list.pop(x)
#         my_list.pop(x)
#         my_list.pop(x+1)
#         print ( my_list )
#         rounds[x] = { "test":[genes[x], genes[x+1]] , "train":my_list}
        rounds[x] = { "test":[genes[x] ], "train":my_list}
#         y = df[[genes[x], genes[x+1]]]

In [14]:
rounds[0]["train"]

['cleavedCas',
 'CyclinB',
 'GAPDH',
 'IdU',
 'Ki.67',
 'p.4EBP1',
 'p.Akt.Ser473.',
 'p.AKT.Thr308.',
 'p.AMPK',
 'p.BTK',
 'p.CREB',
 'p.ERK',
 'p.FAK',
 'p.GSK3b',
 'p.H3',
 'p.HER2_c',
 'p.JNK',
 'p.MAP2K3',
 'p.MAPKAPK2',
 'p.MEK',
 'p.MKK3.MKK6',
 'p.MKK4',
 'p.NFkB',
 'p.p38',
 'p.p53',
 'p.p90RSK',
 'p.PDPK1',
 'p.PLCg2_c',
 'p.RB',
 'p.S6',
 'p.S6K',
 'p.SMAD23',
 'p.SRC',
 'p.STAT1',
 'p.STAT3',
 'p.STAT5']

In [15]:
rounds[0]["test"]

['b.CATENIN']

In [16]:
X = df2[rounds[0]["train"]]
y = df2[rounds[0]["test"]]

In [17]:
%%time
print ("scaling")
scaler = StandardScaler()
scaler.fit(X)
scaled_data = scaler.transform(X)

scaling
CPU times: user 2.88 s, sys: 246 ms, total: 3.12 s
Wall time: 11.3 s


In [18]:
print ("split")
X_train, X_test, y_train, y_test = train_test_split(scaled_data,y,test_size=0.3, random_state=101)

split


In [19]:
# For quick response
n_examples = 4 * len(X_train)
n_params = 8

# In practice, HyperbandSearchCV is most useful for longer searches
# n_examples = 15 * len(X_train)
# n_params = 15

In [20]:
max_iter = n_params  # number of times partial_fit will be called
chunks = n_examples // n_params  # number of examples each call sees

max_iter, chunks

(8, 1240903)

In [45]:
# import dask.array as da
len(X_test.columns)

36

In [43]:
my_x_tst_cols = X_test.columns

In [35]:
X_train2 = X_train.to_dask_array(lengths=True)
y_train2 = y_train.to_dask_array(lengths=True)
X_test2 = X_test.to_dask_array(lengths=True)

In [44]:
X_train2[0]

dask.array<getitem, shape=(36,), dtype=float64, chunksize=(36,), chunktype=numpy.ndarray>

In [36]:
# X_train2.rechunk(chunks=chunks)
# y_train2.rechunk(chunks=chunks)
# X_test2.rechunk(chunks=chunks)
y_train2

dask.array<values, shape=(2481807, 1), dtype=float64, chunksize=(122691, 1), chunktype=numpy.ndarray>

In [ ]:
my_colums  = y_train.columns

In [ ]:
my_index = y_train.index

In [ ]:
len(y_train)/

In [23]:
## params without booster 
params = {'objective': 'reg:squarederror',
        'num_rounds': 100,
        'max_depth':8,
         'max_leaves': 2**8,
#           'nround': 1000,
#           'max_depth': 16,
          'eta': 0.01, 
          'subsample': 0.5,
          'min_child_weight': 1

         }
params['eval_metric'] = ['rmse', 'gamma-nloglik']
# params['booster']='gblinear'

In [24]:

params = { 
'booster':'gblinear',
          'lambda' : 3,
#           'lambda_bias' : 1,
          'gamma' :3,
    'feature_selector':'shuffle',
#         'early_stopping_rounds' : 3
          'num_rounds': 100000,
#           'nround': 1000,
#           'max_depth': 8,
          'max_leaves': 2**8,
#           'eta': 0.01, 
          'subsample': 0.5,
#           'min_child_weight': 1
         }
# params['eval_metric'] = ['rmse', 'gamma-nloglik']
# params['booster']='gblinear'

In [25]:
params = {
"base_score":1, 
"booster":"gblinear", 
"colsample_bylevel":1,
"colsample_bynode":1, 
"colsample_bytree":1, 
"gamma":0,
"importance_type":"gain", 
"learning_rate":0.1, 
"max_delta_step":0,
"max_depth":10, 
"min_child_weight":1, 
"missing":None, 
"n_estimators":1000,
"n_jobs":-1, 
"nthread":None, 
"objective":"reg:squarederror", 
"random_state":101,
"reg_alpha":0, 
"reg_lambda":1, 
"scale_pos_weight":1, 
"seed":0,
"silent":False, 
"subsample":1}
params['eval_metric'] = ['rmse', 'gamma-nloglik']

In [ ]:
params

In [ ]:
# evallist = [(y_train, 'eval'), (X_train, 'train')]

In [37]:
%%time

bst = dxgb.train(client, params, X_train, y_train)
# bst = dxgb.train(client, params,)
bst

CPU times: user 2.32 s, sys: 216 ms, total: 2.53 s
Wall time: 16.2 s


In [ ]:
# fig, ax = plt.subplots(figsize=(12, 8))

# ax = xgb.plot_importance(bst, ax=ax, height=0.8, max_num_features=20)
# ax.grid("off", axis="y")

In [38]:
predictions = dxgb.predict(client, bst, X_test2)
predictions = client.persist(predictions)
predictions

dask.array<_predict_part, shape=(1063621,), dtype=float32, chunksize=(52768,), chunktype=numpy.ndarray>

In [39]:
type(predictions)

dask.array.core.Array

In [40]:
predictions.compute()

ValueError: feature_names mismatch: ['cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.HER2_c', 'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB', 'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.PLCg2_c', 'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35']
expected p.p38, p.MKK3.MKK6, cleavedCas, p.MEK, p.MAP2K3, p.S6, p.Akt.Ser473., p.CREB, p.SRC, p.NFkB, p.AKT.Thr308., p.p90RSK, p.H3, p.4EBP1, p.GSK3b, p.STAT5, p.STAT3, p.FAK, p.STAT1, GAPDH, p.AMPK, p.p53, p.ERK, Ki.67, CyclinB, p.PDPK1, p.MAPKAPK2, p.RB, p.JNK, p.S6K, p.BTK, p.MKK4, IdU, p.PLCg2_c, p.HER2_c, p.SMAD23 in input data


In [29]:
p = predictions.to_dask_dataframe()

In [30]:
type(p)

dask.dataframe.core.Series

In [32]:
p.head().compute()

ValueError: feature_names mismatch: ['cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.HER2_c', 'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB', 'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.PLCg2_c', 'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35']
expected p.p38, p.MKK3.MKK6, cleavedCas, p.MEK, p.MAP2K3, p.S6, p.Akt.Ser473., p.CREB, p.SRC, p.NFkB, p.AKT.Thr308., p.p90RSK, p.H3, p.4EBP1, p.GSK3b, p.STAT5, p.STAT3, p.FAK, p.STAT1, GAPDH, p.AMPK, p.p53, p.ERK, Ki.67, CyclinB, p.PDPK1, p.MAPKAPK2, p.RB, p.JNK, p.S6K, p.BTK, p.MKK4, IdU, p.PLCg2_c, p.HER2_c, p.SMAD23 in input data


In [ ]:
with ProgressBar():
    print ("R^2:",r2_score(y_test.compute(), predictions.compute()))
    print ("MAE:",mean_absolute_error(y_test.compute(), predictions.compute()))
    print ("MSE:",mean_squared_error(y_test.compute(), predictions.compute()))

In [ ]:
# error = predictions.compute() - y_test.compute()
# plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error [MAE]")
# _ = plt.ylabel("Count")
# plt.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    

plt.scatter(y_test.compute(), predictions.compute())
plt.xlabel('True Values [MAE]')
plt.ylabel('Predictions [MAE]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.savefig("MAE_base_model_001.png",format="png")
# plt.savefig("MAE_base_model_%s.png"%(key),format="png")


In [46]:
model = dxgb.XGBRegressor(objective='reg:squarederror',verbosity=3,n_jobs=-1,n_estimators=100)

In [47]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=3)

In [48]:
# model = dxgb.XGBRegressor(base_score=0.5, 
# booster='gbtree', 
# colsample_bylevel=0.8,
# colsample_bynode=0.8, 
# colsample_bytree=0.8, 
# gamma=2,
# importance_type='gain', 
# learning_rate=1, 
# max_delta_step=5,
# max_depth=1, 
# min_child_weight=1, 
# missing=None, 
# n_estimators=100,
# n_jobs=-1, 
# nthread=None, 
# objective='reg:squarederror', 
# random_state=0,
# reg_alpha=1e-2, 
# reg_lambda=1e-2, 
# scale_pos_weight=1, 
# seed=None,
# silent=None, 
# subsample=1,
# verbosity=3
#                 )
# model

In [49]:
%%time
with joblib.parallel_backend('dask'):
    model.fit(X_train2, y_train2)

CPU times: user 50.9 s, sys: 4.69 s, total: 55.6 s
Wall time: 7min 22s


In [ ]:
# X_test = X_test.to_dask_array()
# y_test = y_test.to_dask_array()

In [51]:
predictions = model.predict( X_test2)
predictions = client.persist(predictions)
predictions

dask.array<_predict_part, shape=(1063621,), dtype=float32, chunksize=(52768,), chunktype=numpy.ndarray>

In [52]:
%%time
print ("R^2:",r2_score(y_test.compute(), predictions.compute()))
print ("MAE:",mean_absolute_error(y_test.compute(), predictions.compute()))
print ("MSE:",mean_squared_error(y_test.compute(), predictions.compute()))

R^2: 0.2201942444425512
MAE: 0.41596531638541084
MSE: 0.3089857410261049
CPU times: user 1.62 s, sys: 504 ms, total: 2.12 s
Wall time: 4.46 s


In [60]:
p = predictions.to_dask_dataframe(columns=y.columns)

In [61]:
type(p) , type(predictions)

(dask.dataframe.core.DataFrame, dask.array.core.Array)

In [63]:
p.to_csv("my_result.csv")

['/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/00.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/01.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/02.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/03.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/04.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/05.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/06.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/07.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/08.part',
 '/Users/barradd/Documents/BARRADD_Things/DREAM_challenge/CANCER_challenge/my_result.csv/09.part',
 '/Users/b

In [ ]:
# error = predictions.compute() - y_test.compute()
# plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error [MAE]")
# _ = plt.ylabel("Count")
# plt.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    

plt.scatter(y_test.compute(), predictions.compute())
plt.xlabel('True Values [MAE]')
plt.ylabel('Predictions [MAE]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
# plt.savefig("MAE_base_model_%s.png"%(key),format="png")
plt.savefig("MAE_base_model_002.png",format="png")


In [ ]:
# model = xgb.XGBRFRegressor(objective='reg:squarederror',verbosity=3,n_jobs=-1)
model = xgb.XGBRFRegressor(verbosity=3,objective='reg:squarederror',n_jobs=-1,n_estimators=100)

In [ ]:
%%time
with joblib.parallel_backend('dask'):
    model.fit(X_train2, y_train2)

In [ ]:
# model.save_model("model.0001")
predictions = model.predict( X_test.compute())
# predictions = client.persist(predictions)
predictions

In [ ]:
print ("R^2:",r2_score(y_test.compute(), predictions))
print ("MAE:",mean_absolute_error(y_test.compute(), predictions))
print ("MSE:",mean_squared_error(y_test.compute(), predictions))

In [ ]:
# error = predictions.compute() - y_test.compute()
# plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error [MAE]")
# _ = plt.ylabel("Count")
# plt.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    

plt.scatter(y_test.compute(), predictions)
plt.xlabel('True Values [MAE]')
plt.ylabel('Predictions [MAE]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
# plt.savefig("MAE_base_model_%s.png"%(key),format="png")
plt.savefig("MAE_base_model_002.png",format="png")


#for tuning parameters
parameters_for_testing = {
base_score=[0.5,0.75,1], 
booster='gbtree', 
colsample_bylevel=[0.4,0.6,0.8,1],
colsample_bynode=[0.4,0.6,0.8,1], 
colsample_bytree=[0.4,0.6,0.8,1], 
gamma=[0,0.03,0.1,0.3],
importance_type='gain', 
learning_rate=[0.1,0.01,0.001], 
max_delta_step=[1,5,10],
max_depth=[3,5,7], 
min_child_weight=[1,1.5,6,10], 
missing=None, 
n_estimators=[100,1000,10000],
n_jobs=-1, 
nthread=None, 
objective='reg:squarederror', 
random_state=0,
reg_alpha=[1e-5, 1e-2,  0.75], 
reg_lambda=[1e-5, 1e-2,  0.75], 
scale_pos_weight=1, 
seed=None,
silent=None, 
subsample=[0.5,0.75,1]
}

In [ ]:
# random_grid = {
# "base_score":[0.5,0.75,1], 
# # "booster":"gblinear", 
# "colsample_bylevel":[0.4,0.6,0.8,1],
# "colsample_bynode":[0.4,0.6,0.8,1], 
# "colsample_bytree":[0.4,0.6,0.8,1], 
# "gamma":[0,0.03,0.1,0.3],
# # "importance_type":"gain", 
# "learning_rate":[0.1,0.01,0.001], 
# "max_delta_step":[1,5,10],
# "max_depth":[3,5,7], 
# "min_child_weight":[1,1.5,6,10], 
# # "missing":None, 
# "n_estimators":[100,1000,10000],
# # "n_jobs":-1, 
# # "nthread":None, 
# # "objective":"reg:squarederror", 
# # "random_state":101,
# "reg_alpha":[1e-5, 1e-2,  0.75], 
# "reg_lambda":[1e-5, 1e-2,  0.75], 
# # "scale_pos_weight":1, 
# # "seed":0,
# # "silent":False, 
# # "subsample":1
# }

In [ ]:
random_grid = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0,0.03,0.1,0.3],
   'min_child_weight':[1.5,6,10],
   'learning_rate':[0.1,0.07],
   'max_depth':[3,5],
   'n_estimators':[10000],
   'reg_alpha':[1e-5,
                1e-2,  0.75],
   'reg_lambda':[1e-5, 1e-2, 0.45],
   'subsample':[0.6,0.95]  
}

In [ ]:
model = dxgb.XGBRegressor(objective='reg:squarederror',booster="gblinear",n_jobs=-1)
# model = dxgb.XGBRegressor()

In [ ]:
gsearch1 = RandomizedSearchCV(model,
                              param_distributions = random_grid, 
                              n_jobs=-1,
                              iid=False
#                               scheduler=client                               
#                               scoring='neg_mean_squared_error'
                             )
# cv = KFold(2, shuffle=True)
# gsearch1 = GridSearchCV(estimator = model, 
#                         param_grid = random_grid, 
# #                         cv = cv,
#                         iid=False, 
# #                         random_state=101,
#                         n_jobs = -1,
#                         scoring='neg_mean_squared_error'
#                        )

In [ ]:
import dask.array as da

X_train2 = X_train.to_dask_array(lengths=range(27))
y_train2 = y_train.to_dask_array(lengths=range(27))
# X_train2 = da.from_array(X_train, chunks=chunks)
# X_train2 = X_train.values
# y_train2 = da.from_array(y_train, chunks=chunks)
# y_train2 = y_train.values

In [ ]:
%%time
with joblib.parallel_backend('dask'):
    gsearch1.fit(X_train2, y_train2)

In [ ]:
# print (gsearch1.grid_scores_)
# print('best params')
# print (gsearch1.best_params_)
# print('best score')
# print (gsearch1.best_score_)

In [ ]:
from sklearn.neural_network import MLPRegressor
import numpy as np
model_MLP = MLPRegressor()

In [ ]:
params = {
    "hidden_layer_sizes": [
        (24, ),
        (12, 12),
        (6, 6, 6, 6),
        (4, 4, 4, 4, 4, 4),
        (12, 6, 3, 3),
    ],
    "activation": ["relu", "logistic", "tanh"],
    "alpha": np.logspace(-6, -3, num=1000),  # cnts
    "batch_size": [16, 32, 64, 128, 256, 512],
}

In [ ]:
from dask_ml.model_selection import HyperbandSearchCV

In [ ]:
# # For quick response
# n_examples = 4 * len(X_train)
# n_params = 8

# In practice, HyperbandSearchCV is most useful for longer searches
n_examples = 15 * len(X_train)
n_params = 15

In [ ]:
max_iter = n_params  # number of times partial_fit will be called
chunks = n_examples // n_params  # number of examples each call sees

max_iter, chunks

In [ ]:
search = HyperbandSearchCV(
    model_MLP,
    params,
    max_iter=max_iter,
    patience=True,
)
# search = GridSearchCV(estimator = model_MLP, 
#                         param_grid = params, 
# #                         cv = cv,
#                         iid=False, 
# #                         random_state=101,
#                         n_jobs = -1,
#                         scoring='neg_mean_squared_error'
#                        )

In [ ]:
# search.metadata["partial_fit_calls"]

In [ ]:
X_train2

In [ ]:
%%time
# with joblib.parallel_backend('dask'):
#     gsearch1.fit(X_train, y_train)
search.fit(X_train2, y_train2)